In [42]:
def select_data(db_file_path, db_data, reverse=False, verbose=0):
    import sqlite3
    from contextlib import closing
    with closing(sqlite3.connect(db_file_path)) as conn:
        with closing(conn.cursor()) as cur:
            col_dicts = {"n_estimator": (db_data[1], True),
                         "data_length": (db_data[2], True),
                         "random_state": (db_data[3], True),
                         "rounds": (db_data[4], False)
                         }
            where_list = []
            for col in col_dicts:
                if col_dicts[col][1]:
                    where_list.append(f'{col}={col_dicts[col][0]}')
                else:
                    where_list.append(f'{col}=\"{col_dicts[col][0]}\"')
            results = []
            query = f'select * from rndforest where {" and ".join(where_list)}'
            query += " order by id desc" if reverse else " order by id asc"
            if verbose > 0:
                print('query=', query)
            datas = cur.execute(query).fetchall()
            for data in datas:
                results.append(data)
            return results


def insert_data(db_file_path, db_data, auto_commit=True, verbose=0):
    """ insert_randomforest_db """
    import sqlite3
    from contextlib import closing
    from datetime import datetime
    with closing(sqlite3.connect(db_file_path)) as conn:
        col_dicts = {"n_estimator": db_data[1],
                     "data_length": db_data[2],
                     "random_state": db_data[3],
                     "rounds": db_data[4],
                     "matched_cnts": db_data[5],
                     "sum_val": db_data[6],
                     "version": db_data[7]+"_back"
                     }
        columns = col_dicts.keys()
        values=list(col_dicts.values())
        column_val=["?" for i in range(len(columns))]
        query = f"INSERT INTO rndforest ({','.join(columns)}) "\
        f"VALUES ({','.join(column_val)})"
        with closing(conn.cursor()) as cursor:
            if verbose > 0:
                print(f'query={query}')
                print('-'*30)
            cursor.execute(query, values)
            if auto_commit:
                conn.commit()

def backup_randomforest_db(db_file_path, db_datas, auto_commit=True, verbose=0):
    for db_data in db_datas:
        if verbose > 0:
            print('db_data = ', db_data)
        result = select_data(db_file_path, db_data, verbose=verbose)
        if len(result) == 0:
            insert_data(db_file_path, db_data, verbose=verbose)

In [ ]:
from contextlib import closing
verbose = 0

with closing(open('../db/1.txt', 'r')) as fd:
    print('step.0')
    lines = fd.readlines()
    datas = []
    for line in lines:
        line = line.replace('\n', '')
        datas.append(line.split('|'))
        if verbose > 0:
            print(line.split('|'))
    backup_randomforest_db(db_file_path='../db/metrics.db', db_datas = datas, verbose=1)

In [45]:

!echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like '%3%3%3%3%3%';" | sqlite3 ../db/metrics.db

20944|10|400|338000|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|3,3,1,0,3,0,1,3,3,2|19|T_01_08_back


In [46]:
!echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like '%3%3%3%3%3%';" | sqlite3 ../db/metrics.db

20944|10|400|338000|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|3,3,1,0,3,0,1,3,3,2|19|T_01_08_back


In [57]:
# 3이 많이 발생하는 경우 확인
min_cnt = 3
max_cnt = 7
for i in range(min_cnt, max_cnt):
    !echo "{'-'*15} 3\'s num = {i} {'-'*15}"
    !echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like \"""{'%3'*(i)}""%\" order by sum_val desc" | sqlite3 ../db/metrics.db

--------------- 3\'s num = 3 ---------------
20944|10|400|338000|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|3,3,1,0,3,0,1,3,3,2|19|T_01_08_back
20914|25|400|319350|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,1,3,1,1,3,3,2,0|18|T_01_08_back
20963|10|400|374630|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|3,3,1,2,3,0,2,2,2,0|18|T_01_08_back
20926|25|400|381150|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,1,1,2,0,3,3,2,1|17|T_01_08_back
20961|10|400|373380|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,1,2,0,3,2,3,1,1|17|T_01_08_back
20977|10|400|397710|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,2,1,1,0,2,3,3,1|17|T_01_08_back
20918|25|400|334790|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,2,0,3,0,3,1,1,2|16|T_01_08_back
20921|25|400|348410|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,2,1,3,1,1,3,3,1,0|16|T_01_08_back
20971|10|400|389610|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,1,1,3,0,1,3,3,2,1|16|T_01_08_back
2

In [58]:
# 4가 많이 발생하는 경우 확인
min_cnt = 1
max_cnt = 7
for i in range(min_cnt, max_cnt):
    !echo "{'-'*15} 4\'s num = {i} {'-'*15}"
    !echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like \"""{'%4'*(i)}""%\" order by sum_val desc" | sqlite3 ../db/metrics.db

--------------- 4\'s num = 1 ---------------
20933|10|400|311130|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,2,2,1,2,0,2,4,2,2|18|T_01_08_back
20951|10|400|357530|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,0,2,2,0,1,4,3,2|18|T_01_08_back
20959|10|400|371510|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,2,2,2,1,1,4,2,2,1|18|T_01_08_back
20915|25|400|323230|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,2,1,2,1,0,4,3,2,1|17|T_01_08_back
20920|25|400|342900|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|0,1,1,3,2,2,4,1,3,0|17|T_01_08_back
20924|25|400|366790|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,2,2,2,0,4,0,2,1|17|T_01_08_back
20945|10|400|339240|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,2,1,1,3,2,1,4,1,1|17|T_01_08_back
20954|10|400|360080|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,4,1,1,2,0,2,2,3,1|17|T_01_08_back
20964|10|400|375150|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,3,1,2,2,1,1,4,2,0|17|T_01_08_back
2

In [59]:
# 5가 많이 발생하는 경우 확인
min_cnt = 1
max_cnt = 7
target_num = 5
for i in range(min_cnt, max_cnt):
    !echo "{'-'*15} {target_num}\'s num = {i} {'-'*15}"
    !echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like \"""{'%5'*(i)}""%\" order by sum_val desc" | sqlite3 ../db/metrics.db

--------------- 5\'s num = 1 ---------------
21417|10|400|323450|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,0,0,1,0,0,0,1,5,1|9|T_01_08_back
21418|10|400|361270|1175,1174,1173,1172,1171,1170,1169,1168,1167,1166|1,1,0,0,0,0,5,1,1,0|9|T_01_08_back
--------------- 5\'s num = 2 ---------------
--------------- 5\'s num = 3 ---------------
--------------- 5\'s num = 4 ---------------
--------------- 5\'s num = 5 ---------------
--------------- 5\'s num = 6 ---------------


In [60]:
# 6이 많이 발생하는 경우 확인
min_cnt = 1
max_cnt = 7
target_num = 6
for i in range(min_cnt, max_cnt):
    !echo "{'-'*15} {target_num}\'s num = {i} {'-'*15}"
    !echo "select * from rndforest where version=\"T_01_08_back\" and matched_cnts like \"""{'%6'*(i)}""%\" order by sum_val desc" | sqlite3 ../db/metrics.db

--------------- 6\'s num = 1 ---------------
--------------- 6\'s num = 2 ---------------
--------------- 6\'s num = 3 ---------------
--------------- 6\'s num = 4 ---------------
--------------- 6\'s num = 5 ---------------
--------------- 6\'s num = 6 ---------------
